# Choice of Neighborhood in Toronto for Punjabi Restaurant

One of the largest South-East Asian ethnic groups to live in Canada are Punjabis, hailing from the Northwestern parts of India and Eastern Pakistan. Punjabis first arrived in Canada during the late 19th century to work in the forestry industry. Primarily concentrated in the western province of British Columbia, the Punjabi population initially peaked in 1908 before an ensuing period of population decline and stagnation followed. In the mid-20th-century Canadian immigration laws were relaxed, fostering rapid population growth into the present day. Today, the largest Punjabi communities in Canada are situated in the province of British Columbia, concentrated in Vancouver, and the province of Ontario, particularly in Toronto. The dominant religion of Punjabis, Sikhism is the 7th largest in Canada, accounting for about 1.4% of the total population, roughly 500,000 people as of 2011. But numbers and statistics aside, it is the cultural influence of the Punjabis that has contributed much to the social entropy of the country. Although the largest settlement of Punjabis still remains in British Columbia, By the 1980s, the traditional Punjabi immigration patterns began to shift. Ontario soon became an important center of immigration to Canada. Large Punjabi populations began to appear across the Greater Toronto Area. Later in the 1990s, Alberta became another important immigration destination for Punjabis. Most concentrated in Edmonton and Calgary.
Today, the Punjabi population of Canada is 668,240 with the largest community still located in British Columbia. In addition, Punjabi is the third most spoken language of the Parliament of Canada. So it is quite imperative that there is a burgeoning demand for South-East Asian cuisines across the different neighborhoods of Toronto, and it is this prospect that draws many Punjabi youths from both India and Pakistan to immigrate to Canada, which is also reflected by the data pertaining to worldwide immigration to Canada over the last few decades. So for an aspiring Punjabi youth, qualified in culinary arts or hotel management to open a restaurant in Canada, Toronto is likely to come up as one of the foremost choices. This is where our program comes in, where we can analyze them and visualize the distribution of such region-specific restaurants across different neighborhoods in Toronto so that the person can have a tentative idea of the possible demand that will help him to better decide the final location of his enterprise.

Reference Link:https://theprint.in/theprint-essential/sikhs-so-powerful-in-canada-its-not-about-numbers/314036/
Link to Github file:https://github.com/razim9087/Coursera_Capstone/blob/master/Capstone%20Project.docx

# 1.Description of the dataset used

The data to be used were obtained from Wikipedia, converted to an excel file named List 1.csv which contains a list of different neighborhoods in Toronto pertaining to the listed postal codes and their corresponding boroughs. The geospatial data containing the latitudes and longitudes for the corresponding postal codes have been obtained from an excel file called Geospatial_Coordinates. The combined dataset with both the geospatial and neighborhood datasets was combined using pandas, and the specific neighborhoods of interest were obtained from Foursquare location data using the Foursquare API, to obtain the data regarding various places of interests like restaurants, coffee shops, parks, toy stores, etc. given a certain radius of interest. The areas of interest to be investigated as prospective locations for the restaurant were obtained from the Wikipedia page dedicated to Punjabi Canadians.
Links to referenced datasets or webpages:
Neighborhoods in Toronto: https://github.com/razim9087/Coursera_Capstone/blob/master/List%201.csv
Geospatial_Coordinates: https://github.com/razim9087/Coursera_Capstone/blob/master/Geospatial_Coordinates.csv
Wikipedia Page for Punjabi Canadians: https://en.wikipedia.org/wiki/Punjabi_Canadians
Link to Github file: https://github.com/razim9087/Coursera_Capstone/blob/master/Capstone%20Project.docx

# 2.Code 

In [1]:
#Importing necessary libraries and packages

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
import requests # library to handle requests
from pandas.io.json import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#Importing Toronto Neighbourhood data

df=pd.read_excel("List 1.xlsx")
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#Importing corresponding geo-spatial data

t=pd.read_csv("http://cocl.us/Geospatial_data")
t.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
#Removing postal codes that do not have a borough assogned to

df1=df[df["Borough"]!="Not assigned"].reset_index()
df1.head()

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
#Merging the 2 datasets on Postal Code

df2=pd.merge( df1,t, on='Postal Code', how='outer')
df2.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# 2.1.Creating dataset specific to boroughs of interest

By boroghs of interst, in this case we refer to those boroughs of Toronto that have relatively higher prescence of the Punjabi Community, and our choices are guided by the following wikipedia page(https://en.wikipedia.org/wiki/Punjabi_Canadians) that specifies that Punjabi Canadians are particularly prevalent in the Greater Toronto Area, especially in Scarborough, Markham, Mississauga, Brampton, and Ajax.

In [6]:
#Creating dataset for Scarborough 

neighborhoods_in_Scarborough=df2[df2['Borough']=='Scarborough'].reset_index()
print(neighborhoods_in_Scarborough.shape)
neighborhoods_in_Scarborough.head()

(17, 7)


,level_0,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,6,9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,12,18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,18,27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,22,36,M1G,Scarborough,Woburn,43.770992,-79.216917
4,26,45,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 2.2.Preparing the Foursquare credentials for making API calls

In [7]:
#Foursquare Credentials

CLIENT_ID = '0EAEPMKHBCQDYAMBKH5SILG1QNJ33ZJZJZN5CFNEGY4EDLLS' 
CLIENT_SECRET = 'WEBNEHR3N0UHTDEKVTQZU1KG0VCVEJ2EXPORMKEN031HL4JQ' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0EAEPMKHBCQDYAMBKH5SILG1QNJ33ZJZJZN5CFNEGY4EDLLS
CLIENT_SECRET:WEBNEHR3N0UHTDEKVTQZU1KG0VCVEJ2EXPORMKEN031HL4JQ


# 2.2.Exploring the dataset for specific borough

In [8]:
neighborhoods_in_Scarborough.loc[0, 'Neighbourhood']

'Malvern, Rouge'

In [9]:
neighborhood_latitude = neighborhoods_in_Scarborough.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods_in_Scarborough.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods_in_Scarborough.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


# Querying top 150 Venue around 1Km of Malvern and Rouge of the Scarborough borough using Foursquare API

In [10]:
LIMIT = 150 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0EAEPMKHBCQDYAMBKH5SILG1QNJ33ZJZJZN5CFNEGY4EDLLS&client_secret=WEBNEHR3N0UHTDEKVTQZU1KG0VCVEJ2EXPORMKEN031HL4JQ&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=150'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3aac3b1aa323527dbd86b1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs'

# 2.3.Creating the function that extracts the category of the venue

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# 2.4.Pushing the specifications of the venues obtained in a dataframe

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

17 venues were returned by Foursquare.


,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Harvey's,Restaurant,43.800020,-79.198307
2,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,RBC Royal Bank,Bank,43.798782,-79.197090


# 2.5. Creating function for creating similar queries for all neighborhoods in the borough

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
#Using the getNearbyVenues on the Scarborough neighborhood data

Scarborough_venues = getNearbyVenues(names=neighborhoods_in_Scarborough['Neighbourhood'],
                                   latitudes=neighborhoods_in_Scarborough['Latitude'],
                                   longitudes=neighborhoods_in_Scarborough['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [16]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(93, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [17]:
Scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",8,8,8,8,8,8
"Guildwood, Morningside, West Hill",9,9,9,9,9,9
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


In [18]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


# 3. Analysis of the borough data

In [19]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighbourhood'] = Scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
Scarborough_grouped

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.000000,0.200000,0.000,0.00,0.000,0.000000,0.2,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.2,0.2,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.200000,0.00,0.000,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.25,0.000,0.000000,0.0,0.0000,0.25,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.25,0.000,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.250000,0.00,0.000,0.000000,0.0
2,Cedarbrae,0.000000,0.125,0.125,0.125000,0.000000,0.000000,0.000,0.00,0.125,0.000000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.125000,0.0000,0.125000,0.00,0.000,0.000000,0.125,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.125000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.000,0.083333,0.000000,0.000000,0.000,0.00,0.000,0.083333,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.083333,0.083333,0.0000,0.083333,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.083333,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.083333,0.000000,0.0,0.166667,0.166667,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.083333,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.333333,0.00,0.000,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.000,0.200000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.000,0.4,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.250,0.000000,0.000000,0.000000,0.125,0.00,0.000,0.000000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000000,0.125,0.0,0.125000,0.000000,0.0,0.0,0.0,0.000000,0.125,0.000000,0.00,0.000000,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.125,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.000,0.000,0.111111,0.000000,0.111111,0.000,0.00,0.000,0.000000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.111111,0.000000,0.000000,0.0000,0.000000,0.00,

In [21]:
Scarborough_grouped.shape

(16, 56)

# 3.1.Creating the function to display the top n venues around a neighborhood, n to be specified by the user 

In [22]:
num_top_venues = 5 #input value of n

In [26]:
print("You have opted for viewing "+str(num_top_venues)+" top venues of the borough neighborhoods.")

You have opted for viewing 5 top venues of the borough neighborhoods.


In [27]:
for hood in Scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Breakfast Spot   0.2
2               Skating Rink   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.12
1    Athletics & Sports  0.12
2           Gas Station  0.12
3      Hakka Restaurant  0.12
4   Fried Chicken Joint  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0              Pharmacy  0.17
1           Pizza Place  0.17
2    Italian Restaurant  0.08
3  Fast Food Restaurant  0.08
4          Noodle House  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1 

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False) # to sort the venues in descending order
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
venues_list=list(Scarborough_grouped.columns)
restaurant_list=['Neighbourhood']
i=0
for i in venues_list:
    j='Restaurant' in i
    if j==True:
        restaurant_list.append(i)
        
print(restaurant_list)

['Neighbourhood', 'American Restaurant', 'Caribbean Restaurant', 'Chinese Restaurant', 'Fast Food Restaurant', 'Hakka Restaurant', 'Indian Restaurant', 'Italian Restaurant', 'Korean Restaurant', 'Latin American Restaurant', 'Mexican Restaurant', 'Middle Eastern Restaurant', 'Restaurant', 'Thai Restaurant', 'Vietnamese Restaurant']


In [30]:
Scarborough_grouped_restaurants=Scarborough_grouped[restaurant_list]
Scarborough_grouped_restaurants

,Neighbourhood,American Restaurant,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,Hakka Restaurant,Indian Restaurant,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Restaurant,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.2,0.000000,0.00,0.000000,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0
2,Cedarbrae,0.000000,0.125,0.000000,0.000000,0.125,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.125000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.083333,0.083333,0.000,0.0,0.083333,0.0,0.0,0.000000,0.00,0.000000,0.083333,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.200000,0.000000,0.000,0.4,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.111111,0.00,0.111111,0.000000,0.0
8,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0
9,"Malvern, Rouge",0.000000,0.000,0.000000,1.000000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0


In [31]:
num_top_venues = 5

for hood in Scarborough_grouped_restaurants['Neighbourhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped_restaurants[Scarborough_grouped_restaurants['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1        American Restaurant   0.0
2       Caribbean Restaurant   0.0
3         Chinese Restaurant   0.0
4       Fast Food Restaurant   0.0


----Birch Cliff, Cliffside West----
                  venue  freq
0   American Restaurant   0.0
1  Caribbean Restaurant   0.0
2    Chinese Restaurant   0.0
3  Fast Food Restaurant   0.0
4      Hakka Restaurant   0.0


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.12
1      Hakka Restaurant  0.12
2       Thai Restaurant  0.12
3   American Restaurant  0.00
4    Chinese Restaurant  0.00


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0    Chinese Restaurant  0.08
1  Fast Food Restaurant  0.08
2    Italian Restaurant  0.08
3       Thai Restaurant  0.08
4   American Restaurant  0.00


----Cliffside, Cliffcrest, Scarborough Village West----
                  venue  freq
0   American Restaurant  0.33
1  Car

In [32]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_restaurants_sorted = pd.DataFrame(columns=columns)
neighborhoods_restaurants_sorted['Neighbourhood'] = Scarborough_grouped_restaurants['Neighbourhood']

for ind in np.arange(Scarborough_grouped_restaurants.shape[0]):
    neighborhoods_restaurants_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped_restaurants.iloc[ind, :], num_top_venues)

neighborhoods_restaurants_sorted.head(20)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Latin American Restaurant,Vietnamese Restaurant,Thai Restaurant
1,"Birch Cliff, Cliffside West",Vietnamese Restaurant,Thai Restaurant,Restaurant
2,Cedarbrae,Thai Restaurant,Hakka Restaurant,Caribbean Restaurant
3,"Clarks Corners, Tam O'Shanter, Sullivan",Thai Restaurant,Italian Restaurant,Fast Food Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Vietnamese Restaurant,Thai Restaurant
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant
6,"Golden Mile, Clairlea, Oakridge",Vietnamese Restaurant,Thai Restaurant,Restaurant
7,"Guildwood, Morningside, West Hill",Restaurant,Mexican Restaurant,Vietnamese Restaurant
8,"Kennedy Park, Ionview, East Birchmount Park",Vietnamese Restaurant,Thai Restaurant,Restaurant
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant


# 3.2. Creating Clusters on restaurants data of the concerned borough

In [33]:
# set number of clusters
kclusters = 5

Scarborough_grouped_clustering = Scarborough_grouped_restaurants.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:17] 

array([0, 0, 0, 0, 2, 3, 0, 0, 0, 1, 0, 0, 0, 0, 4, 0])

In [34]:
# add clustering labels
neighborhoods_restaurants_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged = neighborhoods_in_Scarborough

Scarborough_merged = Scarborough_merged.join(neighborhoods_restaurants_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [35]:

Scarborough_merged.head(20)
Scarborough_merged.drop([16],inplace=True)

# 3.3.Creating Visualization

In [65]:
map_clusters = folium.Map(location=[43.7764, -79.2318], zoom_start=12)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighbourhood'], Scarborough_merged['Cluster Labels']):
    text=str(poi) + ' Cluster ' + str(cluster)
    label = folium.Popup(text,parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [87]:
Scarborough_merged.drop(columns=['level_0','index'],inplace=True)

In [88]:
Scarborough_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Vietnamese Restaurant,Thai Restaurant,Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Restaurant,Mexican Restaurant,Vietnamese Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Korean Restaurant,Vietnamese Restaurant,Thai Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Thai Restaurant,Hakka Restaurant,Caribbean Restaurant


In [89]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0, Scarborough_merged.columns[[1] +[2]+ list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",0.0,Vietnamese Restaurant,Thai Restaurant,Restaurant
2,Scarborough,"Guildwood, Morningside, West Hill",0.0,Restaurant,Mexican Restaurant,Vietnamese Restaurant
3,Scarborough,Woburn,0.0,Korean Restaurant,Vietnamese Restaurant,Thai Restaurant
4,Scarborough,Cedarbrae,0.0,Thai Restaurant,Hakka Restaurant,Caribbean Restaurant
5,Scarborough,Scarborough Village,0.0,Vietnamese Restaurant,Thai Restaurant,Restaurant
6,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",0.0,Vietnamese Restaurant,Thai Restaurant,Restaurant
7,Scarborough,"Golden Mile, Clairlea, Oakridge",0.0,Vietnamese Restaurant,Thai Restaurant,Restaurant
9,Scarborough,"Birch Cliff, Cliffside West",0.0,Vietnamese Restaurant,Thai Restaurant,Restaurant
12,Scarborough,Agincourt,0.0,Latin American Restaurant,Vietnamese Restaurant,Thai Restaurant
13,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,Thai Restaurant,Italian Restaurant,Fast Food Restaurant


In [90]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1, Scarborough_merged.columns[[1] +[2]+ list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Scarborough,"Malvern, Rouge",1.0,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant


In [91]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2, Scarborough_merged.columns[[1] +[2]+ list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",2.0,American Restaurant,Vietnamese Restaurant,Thai Restaurant


In [92]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 3, Scarborough_merged.columns[[1] +[2]+ list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
10,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",3.0,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant


In [93]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 4, Scarborough_merged.columns[[1] +[2]+ list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
11,Scarborough,"Wexford, Maryvale",4.0,Middle Eastern Restaurant,Vietnamese Restaurant,Thai Restaurant


# Conclusion with the Scarborough neighborhood

Assuming that none of the restaurants in the list that do not have their speciality specified are Indian restaurants, it is advisable not to choose the neighborhoods of Dorset Park, Wexford Heights, Scarborough Town forsetting up a punjabi restaurant,
since it is likely to be designated as an Indian restaurant by the local populace who do not belong to the Punjabi community. Along with that, the nighbourhoods of Wexford and Maryvale should be preferebly avoided since Middle-East and North-West India shares a taste palette. The most suitable location for the restaurant are chosen to be the Wodburn, West Hill, Guildwood, Morningside and Agincourt since all the restaurants located over there are of completely different genre. 

# 4.Considering neighbourhoods in borough lebelled Toronto following the same steps as above

In [94]:
column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 
neighborhoods_in_Totronto = pd.DataFrame(columns=column_names)
i=0

for i in range(103):
    j="Toronto" in df2.iloc[i]["Borough"]
    if j==True:    
        neighborhoods_in_Totronto = neighborhoods_in_Totronto.append({'Borough':df2.iloc[i]["Borough"] ,
                                          'Neighbourhood': df2.iloc[i]["Neighbourhood"],
                                          'Latitude': df2.iloc[i]["Latitude"],
                                          'Longitude': df2.iloc[i]["Longitude"]}, ignore_index=True)
    i+=1
neighborhoods_in_Totronto .head()    

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


In [96]:
Toronto_venues = getNearbyVenues(names=neighborhoods_in_Totronto['Neighbourhood'],
                                   latitudes=neighborhoods_in_Totronto['Latitude'],
                                   longitudes=neighborhoods_in_Totronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [97]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [98]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


In [99]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0

In [100]:
Toronto_onehot.shape

(1628, 235)

In [101]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.000000,0.017544,0.035088,0.087719,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.017544,0.000000,0.017544,0.000000,0.000000,0.00000,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.0000

In [102]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1            Café  0.04
2     Cheese Shop  0.04
3          Bakery  0.04
4  Farmers Market  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4         Stadium  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.13
1          Restaurant  0.07
2       Burrito Place  0.07
3       Garden Center  0.07
4             Brewery  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4     Boat or Ferry  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1               

In [103]:
Toronto_venues_list=list(Toronto_grouped.columns)
Toronto_restaurant_list=['Neighbourhood']
i=0
for i in Toronto_venues_list:
    j='Restaurant' in i
    if j==True:
        Toronto_restaurant_list.append(i)
        
print(Toronto_restaurant_list)

['Neighbourhood', 'Afghan Restaurant', 'American Restaurant', 'Asian Restaurant', 'Belgian Restaurant', 'Brazilian Restaurant', 'Cajun / Creole Restaurant', 'Caribbean Restaurant', 'Chinese Restaurant', 'Colombian Restaurant', 'Comfort Food Restaurant', 'Cuban Restaurant', 'Doner Restaurant', 'Dumpling Restaurant', 'Eastern European Restaurant', 'Ethiopian Restaurant', 'Falafel Restaurant', 'Fast Food Restaurant', 'Filipino Restaurant', 'French Restaurant', 'German Restaurant', 'Gluten-free Restaurant', 'Greek Restaurant', 'Indian Restaurant', 'Italian Restaurant', 'Japanese Restaurant', 'Korean Restaurant', 'Latin American Restaurant', 'Mediterranean Restaurant', 'Mexican Restaurant', 'Middle Eastern Restaurant', 'Modern European Restaurant', 'Molecular Gastronomy Restaurant', 'Moroccan Restaurant', 'New American Restaurant', 'Persian Restaurant', 'Portuguese Restaurant', 'Ramen Restaurant', 'Restaurant', 'Seafood Restaurant', 'Sushi Restaurant', 'Taiwanese Restaurant', 'Thai Restaura

In [104]:
Toronto_grouped_restaurants=Toronto_grouped[Toronto_restaurant_list]
Toronto_grouped_restaurants

,Neighbourhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Persian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.017544,0.00,0.000000,0.000000,0.000000,0.017544,0.000000,0.00,0.017544,0.000000,0.017544,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.035088,0.035088,0.017544,0.000000,0.017544,0.000000,0.017544,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.00,0.000000,0.015625,0.046875,0.031250,0.015625,0.000000,0.000000,0.000000,0.015625,0.015625,0.000000,0.00000,0.015625,0.000000,0.015625,0.015625,0.000000,0.015625,0.015625,0.000000,0.015625,0.000000,0.015625,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.013158,0.013158,0.000000,0.000000,0.00,0.000000,0.013158,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.013158,0.000000,0.000000,0.000000,0.00,0.000000,0.013158,0.000000,0.065789,0.000000,0.000000,0.026316,0.013158,0.000000,0.000000,0.000000,0.00000,0.000000,0.013158,0.000000,0.013158,0.039474,0.000000,0.052632,0.000000,0.013158,0.013158,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.040000,0.020000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000

In [105]:
num_top_venues = 5

for hood in Toronto_grouped_restaurants['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped_restaurants[Toronto_grouped_restaurants['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0             Seafood Restaurant  0.04
1                     Restaurant  0.04
2            Japanese Restaurant  0.02
3  Vegetarian / Vegan Restaurant  0.02
4             Italian Restaurant  0.02


----Brockton, Parkdale Village, Exhibition Place----
                     venue  freq
0               Restaurant  0.05
1       Italian Restaurant  0.05
2        Afghan Restaurant  0.00
3  New American Restaurant  0.00
4        Korean Restaurant  0.00


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                     venue  freq
0     Fast Food Restaurant  0.07
1               Restaurant  0.07
2        Afghan Restaurant  0.00
3  New American Restaurant  0.00
4        Korean Restaurant  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                       venue  freq
0          Afghan Restaurant   0.0
1         Ita

In [106]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighbourhood'] = Toronto_grouped_restaurants['Neighbourhood']

for ind in np.arange(Toronto_grouped_restaurants.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped_restaurants.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted1.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Seafood Restaurant,Restaurant,French Restaurant,Thai Restaurant,Sushi Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Italian Restaurant,Restaurant,Vietnamese Restaurant,Cuban Restaurant,French Restaurant
2,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Cuban Restaurant,German Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Vietnamese Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant
4,Central Bay Street,Italian Restaurant,Japanese Restaurant,Indian Restaurant,New American Restaurant,French Restaurant


In [107]:
# set number of clusters
kclustersi = 8

Toronto_grouped_clustering = Toronto_grouped_restaurants.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclustersi, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 6, 0, 0, 4, 1, 1, 5, 0])

In [108]:
# add clustering labels
neighborhoods_venues_sorted1.insert(0, 'Cluster', kmeans.labels_)

In [109]:
Toronto_merged = neighborhoods_in_Totronto

Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted1.set_index('Neighbourhood'), on='Neighbourhood')

In [111]:

Toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,French Restaurant,Restaurant,Mexican Restaurant,Vietnamese Restaurant,Cuban Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Italian Restaurant,Sushi Restaurant,Japanese Restaurant,Portuguese Restaurant,Persian Restaurant
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Fast Food Restaurant
3,Downtown Toronto,St. James Town,43.651494,-79.375418,1,American Restaurant,Restaurant,Italian Restaurant,Seafood Restaurant,Moroccan Restaurant
4,East Toronto,The Beaches,43.676357,-79.293031,0,Vietnamese Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant


In [112]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=13)


x = np.arange(kclustersi)
ys = [i + x + (i*x)**2 for i in range(kclustersi)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [113]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 0, Toronto_merged.columns[[0]+[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",French Restaurant,Restaurant,Mexican Restaurant,Vietnamese Restaurant,Cuban Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",Italian Restaurant,Sushi Restaurant,Japanese Restaurant,Portuguese Restaurant,Persian Restaurant
2,Downtown Toronto,"Garden District, Ryerson",Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Ramen Restaurant,Fast Food Restaurant
4,East Toronto,The Beaches,Vietnamese Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant
6,Downtown Toronto,Central Bay Street,Italian Restaurant,Japanese Restaurant,Indian Restaurant,New American Restaurant,French Restaurant
9,West Toronto,"Dufferin, Dovercourt Village",Middle Eastern Restaurant,Vietnamese Restaurant,Doner Restaurant,German Restaurant,French Restaurant
10,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",Restaurant,Italian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant
17,East Toronto,Studio District,American Restaurant,Comfort Food Restaurant,Thai Restaurant,Seafood Restaurant,Italian Restaurant
18,Central Toronto,Lawrence Park,Vietnamese Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant
19,Central Toronto,Roselawn,Vietnamese Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Filipino Restaurant


In [114]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 1, Toronto_merged.columns[[0]+[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Downtown Toronto,St. James Town,American Restaurant,Restaurant,Italian Restaurant,Seafood Restaurant,Moroccan Restaurant
5,Downtown Toronto,Berczy Park,Seafood Restaurant,Restaurant,French Restaurant,Thai Restaurant,Sushi Restaurant
8,Downtown Toronto,"Richmond, Adelaide, King",Restaurant,Thai Restaurant,American Restaurant,Sushi Restaurant,Modern European Restaurant
11,West Toronto,"Little Portugal, Trinity",Asian Restaurant,Vietnamese Restaurant,Restaurant,New American Restaurant,Greek Restaurant
13,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant
16,Downtown Toronto,"Commerce Court, Victoria Hotel",Restaurant,American Restaurant,Seafood Restaurant,Thai Restaurant,Asian Restaurant
27,Downtown Toronto,"University of Toronto, Harbord",Restaurant,Japanese Restaurant,Comfort Food Restaurant,Italian Restaurant,Chinese Restaurant
31,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Vietnamese Restaurant,American Restaurant,Sushi Restaurant,Restaurant,Cuban Restaurant
34,Downtown Toronto,Stn A PO Boxes,Italian Restaurant,Japanese Restaurant,Seafood Restaurant,Restaurant,Molecular Gastronomy Restaurant
36,Downtown Toronto,"First Canadian Place, Underground city",Japanese Restaurant,Restaurant,American Restaurant,Asian Restaurant,Seafood Restaurant


In [115]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 2, Toronto_merged.columns[[0]+[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
21,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",Sushi Restaurant,Vietnamese Restaurant,Cuban Restaurant,German Restaurant,French Restaurant


In [116]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 3, Toronto_merged.columns[[0]+[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Restaurant,Sushi Restaurant,Caribbean Restaurant


In [117]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 4, Toronto_merged.columns[[0]+[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Downtown Toronto,Christie,Italian Restaurant,Restaurant,Vietnamese Restaurant,Cuban Restaurant,French Restaurant
14,West Toronto,"Brockton, Parkdale Village, Exhibition Place",Italian Restaurant,Restaurant,Vietnamese Restaurant,Cuban Restaurant,French Restaurant
15,East Toronto,"India Bazaar, The Beaches West",Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Restaurant,Vietnamese Restaurant
25,West Toronto,"Parkdale, Roncesvalles",Cuban Restaurant,Italian Restaurant,Restaurant,Eastern European Restaurant,Vietnamese Restaurant
35,Downtown Toronto,"St. James Town, Cabbagetown",Restaurant,Italian Restaurant,Chinese Restaurant,Indian Restaurant,Thai Restaurant


In [118]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 5, Toronto_merged.columns[[0]+[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
26,Central Toronto,Davisville,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Greek Restaurant,Thai Restaurant
28,West Toronto,"Runnymede, Swansea",Italian Restaurant,Sushi Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Latin American Restaurant


In [119]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 6, Toronto_merged.columns[[0]+[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,Central Toronto,"North Toronto West, Lawrence Park",Fast Food Restaurant,Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant
38,East Toronto,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Cuban Restaurant,German Restaurant


In [120]:
Toronto_merged.loc[Toronto_merged['Cluster'] == 7, Toronto_merged.columns[[0]+[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,West Toronto,"High Park, The Junction South",Thai Restaurant,Mexican Restaurant,Italian Restaurant,Fast Food Restaurant,Restaurant


# Conclusion with Toronto neighbourhood

To begin with, this section encompasses a larger area than that covered in the previous section. Hence we are going to analyse the results clusterwise, starting with cluster 7, covering the neighborhoods of High Park and The Junction South. Among the top 5 venues, there are virtually no restaurants that are anything like the taste palette of Punjabi food, the closest being Mexican food whic as we all know tend to be very spicy.Same goes for cluster 6 and cluster 2 and 3. Going on to cluster 5, the Davisville neighbourhood will probably not be a good choice given the 2nd most common venue. Same goes for the 4th most common venue of Runnymeade and Swansea neighborhoods where the vegan restaurant is likely to be preferred by the Sikh community who make up the largest section of the Punjabi diaspora. In the 4th cluster all the neighborhoods but St. James Town and Cabbagetown may be considered as suitable restaurant spots. Cluster 1 is also comprised of mostly suitable neighborhoods barring Commerce Court, Victoria Hotel, Church and Wellesly. As for Cluster 0, given the large number of spots that it encompasses, we had to resort to a code in order to have a list of suitable spots for a Punjabi restaurant, that is as follows: 